# Colab Installation and Testing of `torch-geometric-temporal`
This is an edited version of `pytgt_no_permanent_env.ipynb`. It installs the necessary packages to `Drive` and loads them from `Drive`. This prevents us from having to download and install packages every time we want to train a geometric model (~60 min -> ~15 min loading time).

Installation Size: ~4 GB

### Install Packages Permanently
Following https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab.


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/taxi_colab', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


 ### Install Packages (~60 Min)
 - Code follows link: https://stackoverflow.com/questions/67285115/building-wheels-for-torch-sparse-in-colab-takes-forever.

 - A target path is specified for future loading.

 - Note: There is a dependency issue with `pandas` and there may be issues due to installing `numpy==2.0.0`.

 - The following thread might be useful if we have issues associated with CUDA-compatibility: https://github.com/pyg-team/pytorch_geometric/issues/1876.

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install --target=$nb_path numpy==1.25.2 # avoid having to deal with 2.0.0
!pip install --target=$nb_path -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install --target=$nb_path -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install --target=$nb_path -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --target=$nb_path torch-geometric-temporal

2.3.1+cu121


In [ ]:
# if torch-geometric-temporal is not saved in desired directory after the above cell finishes running, try the line below
# this prevents the installation from updating numpy==2.0.0
# and installing torch-scatter, torch-sparse, torch-geometric again (if you have disconnected the runtime)

# !pip install --target=$nb_path -U --no-deps torch-geometric-temporal


### Import packages

If you try to import torch_geometric_temporal, you will get the following error:
```
---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<ipython-input-1-487a31dbc1da> in <cell line: 2>()
      1 import torch_geometric
----> 2 import torch_geometric_temporal

3 frames
/usr/local/lib/python3.10/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py in <module>
      4 import torch.nn as nn
      5 from torch.autograd import Variable
----> 6 from torch_geometric.utils.to_dense_adj import to_dense_adj
      7 import torch.nn.functional as F
      8

ModuleNotFoundError: No module named 'torch_geometric.utils.to_dense_adj'
```
To fix this, go to `/notebooks/torch_geometric_temporal/nn/attention/tsagcn.py` (or `/usr/local/lib/python3.10/dist-packages/torch_geometric_temporal/nn/attention/tsagcn.py` if not installing permanently) and make the following modification:

```
from torch_geometric.utils.to_dense_adj import to_dense_adj
-> from torch_geometric.utils import to_dense_adj
```

Be sure to restart session once you implemented the change.

In [ ]:
# instant if installed, ~10 min if loading from Drive
import torch_geometric
import torch_geometric_temporal

### Benchmark Discrete Datasets (Adaptation)
Adapted from https://colab.research.google.com/drive/12XxBxArzjFwAsiawDMtbJlf2R1dOoS0e?usp=sharing#scrollTo=oObun0dzQHK6.

Later found that it is really just the following tutorial:
https://pytorch-geometric-temporal.readthedocs.io/en/latest/notes/introduction.html#applications.

Code there is not compatible with new version of `torch_geometric_temporal`.

In [ ]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader

loader = ChickenpoxDatasetLoader()
#get_dataset is the method of StaticGraphDiscreteSignal object
dataset = loader.get_dataset()

### Load Train-Test Split
Note, this might not be the same splitting used in the original notebook

In [ ]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal.train_test_split import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

### Build Model

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

### Run Fit

In [ ]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 200/200 [00:56<00:00,  3.53it/s]


In [ ]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 1.0362
